# UETOOLS input files

UETOOLS provides functionality for defining YAML input files. This chapter will focus on the UEDGE-specific setup of YAL input files, and will not discuss the YAML syntax except where it is pertinent to the setup file. There is an abundancy of free YAML tutorials available, such as [this one](https://www.educative.io/blog/yaml-tutorial). UETOOLS uses the "pyaml" package to read the the YAML input files, and applies an internal parses to the loaded data.

## TOC:
* [The input file syntax](#syntax)
    * [Setting one-dimensional arrays](#onedarrays)
        * [Setting whole arrays](#wholearrays)
        * [Setting single elements](#single-elements)
        * [Setting the first N elements](#n-first-elements)
        * [Setting N elements starting from index](#n-elements-from-index)
    * [Variable uniqueness](#uniqueness)
* [Required input decks](#required-decks)
    * [The species input deck](#species-deck)
    * [The grid input deck](#grid-deck)
* [Special variable names](#special-variables)
    * [casename](#casename)
    * [savefile](#savefile)
    * [diff_file](#diff_file)
* [The diffusivity file](#diffusivity-file)
    * [The diff_file structure](#diff-structure)
    * [Variables necessary for isbohmcalc=2](#isbohmcalc2)
    * [Variables necessary for isbohmcalc=0](#isbohmcalc0)
    * [Defining and writing diffusivity files](#defining-diff_files)
        * [Using UETOOLS save-files](#using-save-files)
        * [Writing a dedicated diffusivity file](#dedicated-file)
        * [Calling an external script](#external-script)
* [The "commands" input deck](#commands-deck)
    * [Commands and saves](#commands-saves)
* [Rate files](#rate-files)
* [Example YAML input file](#example-yaml)
* [Converting UETOOLS input files](#converting-input-files)
    * [Writing pure-Python input files](#writing-python)
    * [Writing YAMLS input files from save-files](#writing-yaml)



In [1]:
# Move to the folder containing the parameter scans
from os import chdir
from uetools import Case
chdir('testcase_lores')

UEDGE configuration file /Users/holm10/.uetoolsrc read.


## The input file syntax<a class="anchor" id="syntax"></a>

The UETOOLS input files allow the users to create flexible input files. The input files consist of several "inpur decks", which may be arbitrarily nested and UEDGE variables can be set at any level of the nested tree. The bottom-level identifier must be the name of the UEDGE variable to be set, or one of the special variable names defined below.

    deck1:
        subdeck1:
            variable1: value1
        subdeck2:
            subsubdeck1:
                variable2: value2
    deck2:
        variable3: value3
        
UETOOLS will automatically traverse the structure to the bottom, capture the value and identifier and set them. Thus, there is no restrictions on the input deck names, as these do not cause clashes with variable names in UEDGE. The input file setup is stored as variable Case.varinput['setup'].

### Setting one-dimensional input arrays<a class="anchor" id="onedarrays"></a>
 
In the cases where the variable to be set is multi-dimensional (e.g. isnicore, ncore, etc.), UETOOLS accepts the following syntaxes, treated as follows:

#### Setting the whole array<a class="anchor" id="wholearrays"></a>
The whole 1D array can be set to the same value using:

    variable: value
This is executed in Python as:

    variable = value
    
Example:
    
    isnicore: 1
#### Setting a single element<a class="anchor" id="single-elements"></a>
A single element of the 1D array can be set using:

    variable:
        ind: value
This is executed in Python as:

    variable[ind] = value
    
Example:

    isnicore:
        0: 1
#### Setting the first N elements<a class="anchor" id="n-first-elements"></a>
The first N elements in an array can be set using:

    variable: [value1, value2, ... , valueN]
This is executed in Python as:

    variable[:N+1] = [value1, value2, ... , valueN]    
Example:
    
    isnicore: [1, 0, 0]
#### Setting N elements starting from index<a class="anchor" id="n-elements-from-index"></a>
The N elements following a certain index can be set using:

    variable:
        ind: [value1, value2, ... , valueN]
This is executed in Python as:

    variable[ind:ind+N+1] = [value1, value2, ... , valueN]    
Example:

    isnicore:
        2: [1, 1, 3]
        
### Variable uniqueness<a class="anchor" id="uniqueness"></a>
The input file syntax only makes a single requirement on uniqueness of the variables in the input file: each variable must be specified once only in each input deck. That is, a variable may be defined in multiple decks and/or subdecks (and/or subsubdecks, etc), but not more than once at the same level. As stated above, there are no restrictions on the input deck identifiers, either.

## Required input decks <a class="anchor" id="required-decks"></a>
Although UETOOLS is designed to give the user maximum flexibility, there are two subdecks that must be present in the input file: species and grid. These input decks are necessary as they define the problem size. Any modifications/changes affecting the problem size must be included in these two input decks. Modifications affecting the species/domain but do not alter their size can be included in other input decks, but this is not recommended.

### The species input deck<a class="anchor" id="species-deck"></a>
This input deck is used to define the total number of species present in a UEDGE simulation, which is necessary to allocate the correct memory for derived arrays. The species input deck requires the following variables:

    species:
        # Always mandatory variables
        nhsp: 2         # Number of hydrogen species in simulation
        ngsp: 2         # Number of gaseous species in simulation
        # Variables mandatory if used
        isimpon: 6      # The impurity model used - mandatory if it affects the derived nisp parameter
        nzsp:
            0: 6        # The number of impurity species if isimpon >= 6
            
Note that derived variables, such as nhgsp and nisp do not need to be defined in the species input deck.

### The grid input deck<a class="anchor" id="grid-deck"></a>
The grid input deck is used to specify the size of the numerical domain, which is also necessary to allocate the UEDGE arrays. The grid input deck requires the following variables:

    grid:
        # Always required
        mhdgeo: 1                   # Specifies the magnetic geometry
        geometry: "snull"           # Specifies the MHD equilibrium type
        # Mandatory if used
        gengrid: 0                  # Specifies a grid file to be read rather than created
        isnonog: 1                  # Specifies a nonorthogonal grid and that such a stencil should be applied
        GridFileName: "gridue.hdf5" # Spefifies the grid file to be read
        isgriduehdf5: 1             # Specifies whether the grid file is of HDF5 format
        ...
Any variables affecting the grid size should be included in this block. However, UEDGE contains many options for geometry, grids, and even options to generate grids which are not discussed here. Instead, users are referred to the UEDGE documentation for more information. 

**NOTE:** The recommendation is to read grid-files when using UETOOLS, rather than generating the grids when evaluating the case. This helps isolate physics issues from grid generation issues.

## Special (reserved) variable names<a class="anchor" id="special-variables"></a>

There are a number of special variables that can be defined in the UETOOLS input file, which triggers special actions by UETOOLS. These are typically sections of code which are conditional, or require setting large 2D arrays, which is unfeasable in YAML files. A list and descritpion of these are given below.

### casename<a class="anchor" id="casename"></a>
The variable "casename" is used to give your case an identifier. "casename" can be defined in the input file or upon the creation of a Case object using the kwarg casename. If defined both in the input file and using kwarg casename: in such situations, casename defined using the kwarg takes precedence. "casename" is saved to UETOOLS save files, but only used as an identifier of the case. The identfier is not used by UETOOLS, but is included in the identifier string on figures plotted by UETOOLS.


In [2]:
c1 = Case("input.yaml")
print("Casename from input file: ", c1.info['casename'])
c2 = Case("input.yaml", casename="Manual case name")
print("Casename from Case creation: ", c2.info['casename'])


UETOOLS-style save successfully restored from /Users/holm10/Documents/fusion/uedge/src/UETOOLS/jupyter/testcase_lores/nc20.hdf5

*** UEDGE arrays populated: Case appears converged ***
fnrm without preconditioning: 1.54e+00

Casename from input file:  input
UETOOLS-style save successfully restored from /Users/holm10/Documents/fusion/uedge/src/UETOOLS/jupyter/testcase_lores/nc20.hdf5

*** UEDGE arrays populated: Case appears converged ***
fnrm without preconditioning: 1.54e+00

Casename from Case creation:  Manual case name


### savefile <a class="anchor" id="savefile"></a>
Savefile is the file UETOOLS goes to look for data to restore if the UEDGE option restore=1 is used. In case restore=0, default variables are created and the case is restarted from "scratch" (see UEDGE documentation), and no save-files are required. Using restore=0 is, however, not the recommended way for setting up UEDGE cases, so the "savefile" variable will be used in most of the UEDGE cases.

UETOOLS opens the file specified by savefile and reads and restores the state variables. The savefile location can also be defined upon creation of Case objects using the kwarg savefile. In situations where savefile is defined in both the input file and using the kwarg savefile, the kwarg takes precedence.

In [3]:
c3 = Case("input.yaml")
print("savefile from input: ", c3.info['savefile'])
print("nc20.hdf5 ncore: {}\n\n".format(c3.tools.hdf5search("nc20.hdf5", "ncore")[0]))
c4 = Case("input.yaml", savefile="ncore_down_ncore=1.400e+19_last_ii2.hdf5")
print("savefile from input: ", c3.info['savefile'])
print("ncore_down_ncore=1.400e+19_last_ii2.hdf5: {}\n\n".format(c3.tools.hdf5search("ncore_down_ncore=1.400e+19_last_ii2.hdf5", "ncore")[0]))
print("UEDGE ncore: ", bbb.ncore[0])

UETOOLS-style save successfully restored from /Users/holm10/Documents/fusion/uedge/src/UETOOLS/jupyter/testcase_lores/nc20.hdf5

*** UEDGE arrays populated: Case appears converged ***
fnrm without preconditioning: 1.54e+00

savefile from input:  /Users/holm10/Documents/fusion/uedge/src/UETOOLS/jupyter/testcase_lores/nc20.hdf5
nc20.hdf5 ncore: 2e+19


UETOOLS-style save successfully restored from /Users/holm10/Documents/fusion/uedge/src/UETOOLS/jupyter/testcase_lores/ncore_down_ncore=1.400e+19_last_ii2.hdf5

*** UEDGE arrays populated: WARNING, case NOT converged ***
fnrm without preconditioning: 1.70e+07

savefile from input:  /Users/holm10/Documents/fusion/uedge/src/UETOOLS/jupyter/testcase_lores/nc20.hdf5
ncore_down_ncore=1.400e+19_last_ii2.hdf5: 1.4e+19


UEDGE ncore:  2e+19


From the above example, it can be seen that the save file is restored from the file defined by the kwarg as this takes precedence. Since the restored state is a steady-state solution for a different value of ncore, the creating  the Case using the defaul input-file and restoring a solution for a different setup will, naturally, not be converged.

### diff_file<a class="anchor" id="diff_file"></a>
The final special UETOOLS variable is diff_file, which defines the file to look for particle, heat, and viscous diffusivities in case the UEDGE is set up to use non-constant diffusivities. This file is necesary, as it is not feasible to define 2D arrays, or even long 1D arrays, in YAML input files. UETOOLS looks a HDF5 files containing the diffusivities, according to the diffusivity file description below.


The switch isbohmcalc controls the diffusivity model applied, where isbohmcalc=2 and isbohmcalc=0 specify radial of full 2D diffusivitiy profiles to be used, respectively. In case isbohmcalc=0/2, UETOOLS looks for the required parameters in the file defined by diff_file. For other isbohmcalc options, diff_file is not used.

As with casename and savefile, diff_file can be defined in either the input file or upon creation of Case objects using the kwarg diff_file. In case diff_file is defined by both the input file and and diff_file kwarg, the kwarg takes precedence. 

If diff_file is not defined in eiter the input file or by the diff_file kwarg, and isbohmcalc=0/2, diff_file defaults to savefile, since the diffusivities are written to UETOOLS save files, as detailed below.


## The diffusivity file<a class="anchor" id="diffusivity-file"></a>
Here follows a more detailed description of the diffusivity file, which is useful when trying to make modifications to the diffusivities used by UEDGE. 

### The diff_file structure<a class="anchor" id="diff-structure"></a>
The HDF5 file read by UEDGE should have the following structure:

    <HDF5 file "diff_file" (mode r)>
    └── diffusivities
        ├── <particle diffusivity>
        ├── <electron heat diffusivity>
        ├── <ion heat diffusivity>
        └── <viscous diffusivity>
        
UETOOLS looks for the group "diffusivities" at the top level of the HDF5 file supplied. The "diffusivities" group should contain the necessary datasets. 
        
### Variables necessary for isbohmcalc=2<a class="anchor" id="isbohmcalc2"></a>
When using radially varying diffusivities (option isbohmcalc=2), the following datasets should be present in the "diffusivities" group:

    <HDF5 file "diff_file" (mode r)>
    └── diffusivities
        ├── difniv # Particle diffusivity
        ├── kyev # Electron heat diffusivity
        ├── kyiv # Ion heat diffusivity
        └── travisv # Viscous diffusivity
**NOTE:** Each variable should be a 1D array of length equal to the radial grid size, e.g. ny+2, except difniv which is to be a 2D-array with shape (ny+2, nisp). See UEDGE documentation for further details.


### Variables necessary for isbohmcalc=0<a class="anchor" id="isbohmcalc0"></a>
When using manually defined diffusivities (option isbohmcalc=0), the following datasets should be present in the "diffusivities" group:

    <HDF5 file "diff_file" (mode r)>
    └── diffusivities
        ├── dif_use # Particle diffusivity
        ├── kye_use # Electron heat diffusivity
        ├── kyi_use # Ion heat diffusivity
        ├── tray_use # Viscous diffusivity
        └── fcdif # Optional, but recommended: see below
        
**NOTE:** Each variable should be a 2D array of shape equal to the grid size, e.g. (nx+2, ny+2), except dif_use which is to be a 3D array with shape (nx+2, ny+2, nisp). See UEDGE documentation for further details.


**NOTE:** When using isbohmcalc=0 the total diffusivities evaluated are treated as:

    fcdif*<constant diffusivity> + <user-specified diffusivity>
    
e.g., the actual particle diffusivity used in UEDGE is calculated as

    fcdif*difni + difni_use
    
Thus, it is important to remember to set fcdif=0 when the diffusivities should be set completely manually using the \*\_use arrays.

### Defining and writing diffusivity files<a class="anchor" id="defining-diff_files"></a>
There are a number of ways to define and write diffusivity files for use with UETOOLS, some of which are outlined below.

#### Using UETOOLS save-files<a class="anchor" id="using-save-files"></a>
Utilizing UETOOLS save-files is the recommended way to track diffusivities. Whenever saving a UEDGE case using UETOOLS, the currently defined diffusivities are automatically written to the file in a compatible structure:

In [4]:
c4.tools.hdf5tree("nc20.hdf5", group="diffusivities")

<HDF5 file "nc20.hdf5" (mode r)>
diffusivities
└── bbb
    ├── dif_use (18, 10, 8)
    ├── difni (31)
    ├── difniv (10, 8)
    ├── fcdif (float)
    ├── isbohmcalc (int)
    ├── kye (float)
    ├── kye_use (18, 10)
    ├── kyev (10)
    ├── kyi (float)
    ├── kyi_use (18, 10)
    ├── kyiv (10)
    ├── travis (31)
    ├── travisv (10, 8)
    └── tray_use (18, 10, 8)


As can be seen, the save-file contains all diffusivity-related variables for any isbohmcalc option. This means anytime a converged UEDGE case is saved using UETOOLS, the accompanying diffusivities are written to the same file. Thus, if starting from a previously converged case, modifications can be made to the diffusivity profiles and these will be carried along with the save-files. Thus, diff_file is commonly not defined in the input deck, meaning UETOOLS restores the diffusivities from the save-file. Any modifications made interactively to the UEDGE case in an active UEDGE session will be tracked by the UETOOLS save file. 

In situations where one needs to change the diffusivities of a UEDGE simulation to match that of another UEDGE simulation converged using UETOOLS, the functions Case.radialdiff and Case.userdiff can be used to restore the radial and 2D diffusivities from another UETOOLS save file (or any compatible HDF5 file). This will automatically set the necessary diffusivity profiles.

To only change a subset of the diffusivity profiles, the function Case.hdf5search (see Chapter 9) can be utilized to change the infividual diffusivity coefficients, e.g.:

    bbb.dif_use = c.hdf5search(<filename>, "dif_use")
    
**NOTE:** the case the diffusivity profiles are imported from must have the same dimension arrays as the case they are imported to (e.g. nx, ny, and nisp). There are no checks on the cell distribution (e.g. ixpt1, ixpt2, iysptrx) or cell distribution between cases and are the responsibility of the user. 

#### Writing a dedicated diffusivity file<a class="anchor" id="dedicated-file"></a>
Any external script that produces a compatible diffusivity HDF5 file may be used to produce such a file, which can then be read using diff_file. 

**Note:** the diffusivities read from the external file are re-written to any UETOOLS save-files, meaning there is no need to use an external file after using UETOOLS to write a save file.

#### Calling an external script <a class="anchor" id="external-script"></a>
The "commands" input deck (see below) can be used to call external scripts or functions, which modify the diffusivity coefficients in UEDGE. Since any lines in the "command" input-deck are performed after all other inputs are read and executed, any such scripts/modifications will overwrite the diffusivities read from diff_file etc. 

**Note:** any changes made to the diffusivities in external scripts called from the "command" input-deck are written to the UETOOLS save-file. Thus, the external script only needs to be called when making the intiial changes, after which the input file can be used.

## The "commands" input deck<a class="anchor" id="commands-deck"></a>
The special input deck "commands" can be used in situations when it is necessary to make Python calls as part of the input file, such as when UEDGE is used to pupulate arrays that later are modified. One such example is when setting localized pumping surfaces using the switch albedo_by_use:

    commands:
        - self.populate()
        - bbb.albedo_by_user = 1
        - bbb.albrb[24:30] = 0.9
        
                


The "commands" input-deck consists of a list of pure-Python commands, treated as strings, which are read and executed line-by line. In the above example, Case.populate is called to populate the UEDGE arrays used for recycling, such as albrb and alblb. **Note:** Because the lines in "commands" are parsed internally by the Case object, any calls to methods belonging to Case must be called using self.function rather than by Case.function. The second line tells UEDGE to use the albrb and alblb arrays defining the radial albedo profiles along the right and left boundaries, respectively, as-is rather than populating them based using albedorb and albedolb. Finally, the third line modifies the previously populated arrays to define a pumping surface over a subset of the target cells. 

**NOTE:** The "commands" input deck can, in principle, be used to evaluate confditionals or create loops, but the syntax becomes complicated since each line is executed as a command. That means, each line must contain a full, valid python command, so conditionals or loops must be defined completely on a single line:

**VALID**

    commands:
        - "for i in range(10):\n    print(i)"
**INVALID**

    commands:
        - for i in range(10):\n    print(i)
**INVALID**
        
    commands:
        - "for i in range(10):"
        - "    print(i)"
Thus, it is recommanded to utilize external scripts for executing any complex commands:

In file script.py:
    
    def print_series(N)
        for i in range(N):
            print(i)

In input file:

    commands:
        - from script import print_series
        - print_series(10)
        
### Commands and saves<a class="anchor" id="commands-saves"></a>
Any commands defined in the input file are stored to to the save-file after execution:
    
    <HDF5 file "save.hdf5" (mode r)>
    └── setup
        └── commands
        
When restoring a UEDGE case from a save file using UETOOLS (see Chapter 4), the commands are not executed because the paths to external scripts may be broken, etc. Instead, when restoring a Case from a save file using UETOOLS, the commands are ignored and UETOOLS restores the automatically detected changes, as explained in Chapter 4. Thus, UETOOLS save files should work as self-contained UEDGE cases regardless of the changes done in the "commands" input-deck, since any changes affecting the setup is detected and carried along with the case in the save-file.

## Rate files<a class="anchor" id="rate-files"></a>
The reaction rates used in UEDGE are defined in separate external files. All of these files are not publicly avialable and, thus, cannot be distributed with UEDGE. Therefore, the files used, and location of the files, will be different for the individual users. This section will shortly outline how to define the paths to the different rate files, and what the precedence is between the different methods.

UEDGE looks for the hydrogen and impurity rate files in the locations specified by aphdir and apidir, respectively. The UEDGE default is the current work directory. The paths may be defined in a .uedgerc.py file (see UEDGE documentation). If the aphdir and apidir are defined in .uedgerc.py they will be used unless they are defined in the input file, as this definition takes precedence. 

UETOOLS also offers two options for setting the rate paths. The rate paths can be set in the UETOOLS rc file .uetoolsrc in the user's home directory. This file can be created manually or by calling Case.configcase (see Chapter 1). If .uetoolsrc (which is a single-depth YAML file) contains definitions for aphdir apidir, these definitions will take precedence over the paths defined in .uedgerc.py or in any UETOOLS YAML input file. 

Just like the special variables discussed above, aphdir and apidir can be set upon creation of a Case object by supplying the kwargs aphdir and apidir. Any rate paths defined when creating an object will take precedence over those defined in .uetoolsrc.

However, any external input files will take precedence over all of the above, as re-defining the variables directly in UEDGE memory bypasses UETOOLS completely. The same holds for definitions of aphdir/apidir in the "commands" input deck.

1. External scripts (including calls in the "commands" input deck)
2. Kwargs supplied when create the Case object
3. .uetoolsrc
4. aphdir/apidir-definitions in YAML input files
5. .uedgerc.py

## Example YAML input file<a class="anchor" id="example-yaml"></a>
The example case uses a UETOOLS YAML input file:

In [5]:
cat input.yaml


casename: "testcase"
savefile: "nc20.hdf5"
restart: 1

species:
    ngsp: 2     # No. of gas species
    nhsp: 2     # No. of hydrogenic species
    isimpon: 6  # Turns on impurity model no 6 (force-balance)
    nzsp:       # Array of impurity species
        0: 6    # Turn on one impurity species, 6 charge-states
    
grid:
    mhdgeo: 1   # Toroidal MHD equilibrium
    geometry: "snull" #
    isnonog: 1  # Evaluates nonorthogonal fluxes on grid
    gengrid: 0  # Read grid from file
    GridFileName: "gridue.hdf5" # Path to grid file
    isgriduehdf5: 1 # Switch whether to read HDF5 gridue file

solver:
    svrpkg: "nksol"
    premeth: "ilut"

physics:
    oldseec: 0

boundaryconditions:
    isnicore: 
        0: 1
    ncore: 2.e19
    iflcore: 0
    tcoree: 250
    tcorei: 250
    tedge: 2
    t_wall: 300
    t_plat: 500
    
atoms:
    ziin:
        1: 0
    ineudif: 2
    isupgon:
        0: 1
    isngon:
        0: 0
    cfnidh: 0.2

rates:
    istabon: 10

carbon:
    ngbackg:
 

Because the UEDGE case here is an example case, the case and, consequently, input file is rather simple. When including more physics in simulations, and trying to better match boundary conditions and data from experiments, the input files can grow in size and complexity.
    
## Converting UETOOLS input files<a class="anchor" id="converting-input-files"></a>
When sharing UEDGE case run using UETOOLS with non-UETOOLS users, the UETOOLS YAML input files cannot be directly executed. In fact, there is no established "default format" for UEDGE cases apart from the UETOOLS format. Under such circumstances, stand-alone Python scripts may be used as "universal" input files. Converting a YAML input file to a Python input file can, however, be quite time-consuming. Therefore, UETOOLS offers some functions for expediting this process.


### Writing pure-Python input files<a class="anchor" id="writing-python"></a>
Any UETOOLS input created using UETOOLS from a YAML-style input or from a UETOOLS save-file can be written to a standard Python format using the Case.write_py function:

In [6]:
help(c4.convert.write_py)

Help on method write_py in module uetools.UeUtils.Convert:

write_py(fname) method of uetools.UeUtils.Convert.Convert instance
    Writes a standalone Python input file from Case to fname



Case.write_py attempts to write a standalone Python input file to a file fname. 

In [7]:
c4.convert.write_py("input.py")
!cat input.py

from uedge import *
from h5py import File
from uedge.hdf5 import hdf5_restore

casename = "input"
aph.aphdir = "/Users/holm10/Documents/fusion/uedge/rates/aph"
api.apidir = "/Users/holm10/Documents/fusion/uedge/rates/api"
bbb.restart=1

# ==== SPECIES ====
com.ngsp=2
com.nhsp=2
bbb.isimpon=6
com.nzsp[0]=6

# ==== GRID ====
bbb.mhdgeo=1
com.geometry="snull"
com.isnonog=1
bbb.gengrid=0
bbb.GridFileName="gridue.hdf5"
com.isgriduehdf5=1

# ==== SOLVER ====
bbb.svrpkg="nksol"
bbb.premeth="ilut"

# ==== PHYSICS ====
bbb.oldseec=0

# ==== BOUNDARYCONDITIONS ====
bbb.isnicore[0]=1
bbb.ncore="2.e19"
bbb.iflcore=0
bbb.tcoree=250
bbb.tcorei=250
bbb.tedge=2
bbb.t_wall=300
bbb.t_plat=500

# ==== ATOMS ====
bbb.ziin[1]=0
bbb.ineudif=2
bbb.isupgon[0]=1
bbb.isngon[0]=0
bbb.cfnidh=0.2

# ==== RATES ====
com.istabon=10

# ==== CARBON ====
bbb.ngbackg[1]="1.e9"
bbb.ingb=2
bbb.istgcon[1]=1
bbb.tgas[1]=1
bbb.rcxighg=0
bbb.n0g[1]="1.e16"
bbb.isch_sput[1]=7
bbb.isph_sput[1]=3
bbb.csfaclb[2:8]=[2.191, 2.191, 

The resulting Python script should not be considered a complete stand-alone input file for UEDGE but, rather, a template for converting a YAML input file to a Python script, since there always are risks involved in automated conversion of scripts, such as dimension mismatches etc.

### Writing YAMLS input files from save-files<a class="anchor" id="writing-yaml"></a>
Similarly, there may be a need to generate a UETOOLS YAML input file from a UETOOLS save that has been used in standalone mode. The function Case.write_yaml can be used for this purpose:


In [8]:
help(c4.convert.write_yaml)

Help on method write_yaml in module uetools.UeUtils.Convert:

write_yaml(fname) method of uetools.UeUtils.Convert.Convert instance
    Writes a UETOOLS YAML input file from Case to fname



Similarly to Case.write_py, write_yaml writes a template UETOOLS YAML input file to fname:

In [9]:
c4.convert.write_yaml("test.yaml")
!cat test.yaml

casename: testcase
savefile: nc20.hdf5
restart: 1
species:
  ngsp: 2
  nhsp: 2
  isimpon: 6
  nzsp:
    0: 6
grid:
  mhdgeo: 1
  geometry: snull
  isnonog: 1
  gengrid: 0
  GridFileName: gridue.hdf5
  isgriduehdf5: 1
solver:
  svrpkg: nksol
  premeth: ilut
physics:
  oldseec: 0
boundaryconditions:
  isnicore:
    0: 1
  ncore: 2.e19
  iflcore: 0
  tcoree: 250
  tcorei: 250
  tedge: 2
  t_wall: 300
  t_plat: 500
atoms:
  ziin:
    1: 0
  ineudif: 2
  isupgon:
    0: 1
  isngon:
    0: 0
  cfnidh: 0.2
rates:
  istabon: 10
carbon:
  ngbackg:
    1: 1.e9
  ingb: 2
  istgcon:
    1: 1
  tgas:
    1: 1
  rcxighg: 0
  n0g:
    1: 1.e16
  isch_sput:
    1: 7
  isph_sput:
    1: 3
  csfaclb:
    2: [2.191, 2.191, 2.191, 2.191, 2.191, 2.191]
  csfacrb:
    2: [2.191, 2.191, 2.191, 2.191, 2.191, 2.191]
  minu:
    2: [12, 12, 12, 12, 12, 12]
  ziin:
    2: [1, 2, 3, 4, 5, 6]
  znuclin:
    2: [6, 6, 6, 6, 6, 6]
  n0:
    2: []
  fphysylb:
    1:
      0: 0.5
  fphysyrb:
    1:
      0: 0.5
  fche

Just as with Case.write_py, YAML files written using Case.write_yaml should be considered input file templates rather than functional input files and should be verified. The primary concern is the Python commands in the "commands" input deck and the automatically detected changes in the input file. 

Case.write_yaml does not write the vairables with automatically detected changes to the input files because, as discussed in Chapter 4, UETOOLS detects changes to a number of 2D arrays that can be used as inputs for certain UEDGE setups. Thus, writing all of these variables to an input file would make the file unreadable. Instead, Case.write_yaml reverts to using the Python commands defined in the original input file's "commands" deck. Any changes made to variables that are not listed in the general input decks in the original input file after loading the case from the input file will be lost. Furthermore, any commands calling local scripts may now have broken paths, if the UETOOLS save is no longer in the same location.